In [1]:
with open('api.txt','r') as r:
    TEXT = r.read()

In [2]:
from ecommerce.src_base_scraper.scraper_builder import ScraperManagerBuilder
from ecommerce.src_base_scraper.requests_handler import RequestsHandler
from ecommerce.scraper_data.functions import get_pages,get_product,get_product_data

In [3]:
handler = RequestsHandler.create_instance(None)
builder = ScraperManagerBuilder(requests_handler = handler,
                      initial_url='https://www.dedeman.ro/ro/hote-bucatarie-si-accesorii/c/669',
                      initial_node_function = get_pages,
                      node_functions=[get_product],
                      product_functions = get_product_data
                      )

In [4]:
scraper = builder.build()

In [7]:
a = scraper.get_data(parallel_processing_flag=True,depth_parallelism=3)

In [8]:
for i in a :
    print(i)

ProductData(url='https://www.dedeman.ro/ro/hota-clasica-zanussi-zht531w-220-mc/h-1-motor-latime-50-cm-alba/p/3032950', product_info='Hota clasica Zanussi ZHT531W, 220 mc/h, 1 motor, latime 50 cm, alba', raw_text_response='')
ProductData(url='https://www.dedeman.ro/ro/hota-incorporabila-bosch-dhl885c-730-mc/h-1-motor-277-w-latime-86-cm-argintie/p/3035409', product_info='Hota incorporabila Bosch DHL885C, 730 mc/h, 1 motor, 277 W, latime 86 cm, argintie', raw_text_response='')
ProductData(url='https://www.dedeman.ro/ro/hota-decorativa-bosch-dwb67im50-752-mc/h-1-motor-255-w-latime-60-cm-argintie/p/3035338', product_info='Hota decorativa Bosch DWB67IM50, 752 mc/h, 1 motor, 255 W, latime 60 cm, argintie', raw_text_response='')
ProductData(url='https://www.dedeman.ro/ro/hota-incorporabila-bosch-dwk97jm20-730-mc/h-1-motor-143-w-latime-90-cm-alba/p/3031107', product_info='Hota incorporabila Bosch DWK97JM20, 730 mc/h, 1 motor, 143 W, latime 90 cm, alba', raw_text_response='')
ProductData(url='ht